In [1]:
import pandas as pd; import numpy as np; import matplotlib.pyplot as plt
import datetime as dt
import param
import pandas as pd
from astropy.coordinates import SkyCoord, Angle
from astropy import units as u
import warnings
import logging
from astropy.io import fits
from matplotlib.figure import Figure
from matplotlib import cm
from pathlib import Path
import panel as pn
import param
import datetime as dt
from datetime import date
from astroquery.simbad import Simbad
import astroquery



pn.extension()

In [2]:
class SimbadQuery(param.Parameterized):
        action = param.Action(lambda x: x.param.trigger('action'), label='Search for Object',precedence=0.5)
        #select_object = param.String(default="", label="Enter object name",precedence=-0.4)
        search_coordinates = param.String(default='' , precedence=-0.4)  
        object_or_coordinates = param.Selector(objects=['Object      ','Coordinates'],precedence=0.3)
        select_object = param.String(default='',precedence=0.4)  

        # @param.depends('object_or_coordinates','select_object','search_coordinates',watch=True)
        # def object_coordinates_search(self):
        #     """
        #     Whenever the object option is chosen, the coordinates settings will disappear and vice versa
        #     """
        #     if self.object_or_coordinates == 'Coordinates':
        #         self.param.select_object.precedence = -0.4
        #         self.param.search_coordinates.precedence = 0.4
        #     else:
        #         self.select_object = ''
        #         self.param.select_object.precedence = 0.4
        #         self.param.search_coordinates.precedence = -0.4

        # @param.depends('search_coordinates')
        # def coordinates_to_degrees(self):
        #     """
        #     Users can enter right ascension in hours, minutes and seconds
        #     Users can declination in degrees, minutes and seconds
        #     Here they are transformed to degrees, which can be used to search the database
        #     """
        #     if self.search_coordinates == '':
        #         return
        #     right_ascension, declination = self.search_coordinates.split(',')
        #     if len(right_ascension) > 1:
        #         if float(right_ascension.split(' ')[0]) >= 24:
        #             right_ascension = Angle(right_ascension, unit=u.degree)
        #             right_ascension.wrap_at(360 * u.degree, inplace=True)
        #         else:
        #             right_ascension = Angle(right_ascension, unit=u.hourangle)
        #             right_ascension.wrap_at(24 * u.hourangle, inplace=True)
        #     else:
        #         if float(right_ascension) >= 24:
        #             right_ascension = Angle(right_ascension, unit=u.degree)
        #             right_ascension.wrap_at(360 * u.degree, inplace=True)
        #         else:
        #             right_ascension = Angle(right_ascension, unit=u.hourangle)
        #             right_ascension.wrap_at(24 * u.hourangle, inplace=True)
        #     declination = Angle(declination, unit=u.deg)
        #     right_ascension = right_ascension.deg
        #     declination = declination.deg
        #     if declination < -90:
        #         while declination < -90:
        #             declination += 90
        #     elif declination > 90:
        #         while declination > 90:
        #             declination -= 90
        #     return right_ascension, declination

        @param.depends('action',watch=True)
        def query_simbad(self):
            if self.select_object == '':
                obj = self.search_coordinates.strip()
            else:
                obj = self.select_object.strip()
            if obj:
                with warnings.catch_warnings():
                    warnings.filterwarnings("error")
                    try:
                        star = Simbad.query_object(obj)
                        coordinates = SkyCoord(star['RA'], star['DEC'], unit=(u.hourangle, u.deg))
                        object_ra, object_dec = coordinates.ra.deg[0], coordinates.dec.deg[0]
                        res=pn.pane.Markdown('Object located at RA: {object_ra} and DEC: {object_dec}'.format(object_ra=self.object_ra,object_dec=object_dec )) 
                        return pn.Column(res,margin=(40, 10))
                    except Exception as e:
                        return pn.Column(pn.pane.Markdown('Error found in {err}'.format(err=e)),margin=(40, 10))


            #return pn.Column(self.query_simbad)

In [3]:
class MainApplication(SimbadQuery):
    
    start = dt.date(2023, 8, 6)
    end = dt.date(2025, 12, 15)
    username = param.String(default="",precedence=0)
    rank  = param.ObjectSelector(default="Student", objects=["Moderator", "OA", "Staff","Student", "Student (Thesis)", "Outreach/Schools", 'Public'],precedence=0.1)
    dater = param.Date(default=date.today(), bounds=(start, end),precedence=0.2)
    #action = param.Action(lambda x: x.param.trigger('action'), label='Search for Object',precedence=0.5)
    #object_or_coordinates = param.Selector(objects=['Object      ','Coordinates'],precedence=0.3)
    #select_object = param.String(default='',precedence=0.4)
    #search_coordinates = param.String(default='' , precedence=-0.4)   
    Filters  = param.ListSelector(default=['Blue','Green','Red'],
    objects=['Lum', 'Blue', 'Green', 'Red','H \u03B1', 'S[II]'], precedence=2)
    amounthour= param.Number(default=1, bounds=(0, 12),precedence=0.7)
    IMAGE = pn.pane.JPG('https://cdn.nieuws.nl/media/sites/75/2020/07/20174423/Sterrenwacht-915x518.jpg')
    simbad_query = param.ClassSelector(class_=SimbadQuery)
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        self.simbad_query = SimbadQuery()
    @param.depends('username', 'rank', 'amounthour')
    def view(self):
        tikor=self.amounthour
        status = pn.pane.Markdown('{ranko} : {username} wants to observe for {tikor:.0f} hours  '.format(
            ranko=self.rank,username=self.username,tikor=tikor),)
        return pn.Column(
   status,  margin=(40, 10)
)
    
    @param.depends('object_or_coordinates','select_object','search_coordinates',watch=True)
    def object_coordinates_search(self):
        """
        Whenever the object option is chosen, the coordinates settings will disappear and vice versa
        """
        if self.object_or_coordinates == 'Coordinates':
            self.param.select_object.precedence = -0.4
            self.param.search_coordinates.precedence = 0.4
        else:
            self.select_object = ''
            self.param.select_object.precedence = 0.4
            self.param.search_coordinates.precedence = -0.4


    @param.depends('search_coordinates')
    def coordinates_to_degrees(self):
        """
        Users can enter right ascension in hours, minutes and seconds
        Users can declination in degrees, minutes and seconds
        Here they are transformed to degrees, which can be used to search the database
        """
        if self.search_coordinates == '':
            return
        right_ascension, declination = self.search_coordinates.split(',')
        if len(right_ascension) > 1:
            if float(right_ascension.split(' ')[0]) >= 24:
                right_ascension = Angle(right_ascension, unit=u.degree)
                right_ascension.wrap_at(360 * u.degree, inplace=True)
            else:
                right_ascension = Angle(right_ascension, unit=u.hourangle)
                right_ascension.wrap_at(24 * u.hourangle, inplace=True)
        else:
            if float(right_ascension) >= 24:
                right_ascension = Angle(right_ascension, unit=u.degree)
                right_ascension.wrap_at(360 * u.degree, inplace=True)
            else:
                right_ascension = Angle(right_ascension, unit=u.hourangle)
                right_ascension.wrap_at(24 * u.hourangle, inplace=True)
        declination = Angle(declination, unit=u.deg)
        right_ascension = right_ascension.deg
        declination = declination.deg
        if declination < -90:
            while declination < -90:
                declination += 90
        elif declination > 90:
            while declination > 90:
                declination -= 90
        return right_ascension, declination

    #@param.depends('username', 'rank', 'dater','amounthour', 'search_coordinates', 'select_object','object_or_coordinates')
    def panel(self):
        #IMAGE = pn.pane.JPG('https://cdn.nieuws.nl/media/sites/75/2020/07/20174423/Sterrenwacht-915x518.jpg', width=700, height=500)
        #IMAGE.style = {'border': '5px solid black', 'background-color': 'lightgray'}
        image_url = 'https://cdn.nieuws.nl/media/sites/75/2020/07/20174423/Sterrenwacht-915x518.jpg'
        IMAGE_HTML = pn.pane.HTML(f'<img src="{image_url}" style="border: 4px solid black; background-color: CornflowerBlue;" width="600" height="400">')

        mainwidgets={
            'username':{'widget_type':pn.widgets.TextInput, 'name': 'Name'},
            'rank':{'widget_type':pn.widgets.Select,'name':'Observer type '},
            'dater':{'widget_type':pn.widgets.DatePicker,'name': 'Observing Date'},
            'object_or_coordinates':{'type': pn.widgets.RadioButtonGroup, 'button_type' : 'primary',"align": "start"},
            'select_object':{'widget_type':pn.widgets.TextInput, 'name': 'Name of the Object'},
            'search_coordinates': {'widget_type': pn.widgets.TextInput, 'name': 'Coordinates (hh mm ss.ms, dd mm ss.ms)'},
            'amounthour':pn.widgets.FloatSlider(name='Duration of Observing (Hours)', start=0, end=12, step=0.5)
        }
        return pn.Row(pn.Column(pn.pane.Markdown("# **Observation Request**", sizing_mode="stretch_width", styles={'background': 'lightblue'})
            ,pn.Param(self,widgets=mainwidgets,show_name=False,sizing_mode="stretch_width",styles={'background': 'lightblue'}),self.view,),IMAGE_HTML,)


In [4]:
class MainApplic(MainApplication):
    simbad_query = param.ClassSelector(class_=SimbadQuery)
    Features=param.ClassSelector(class_=MainApplication)
    def __init__(self):
        super().__init__()
        self.simbad_query = SimbadQuery()
        self.Features=MainApplication()

    def panel(self):
        # Create the layout for the combined app here, including both main_app and simbad_query widgets
        main_lay = self.Features.panel()
        #simbad_query_layout = self.simbad_query.panel()

        # Combine the layouts as needed
        combined_layout = pn.Column(main_lay) #,simbad_query_layout)

        return combined_layout
app=MainApplic()

main_layout = app.panel()
main_layout.servable()
# Serve the main application layout



Column
    [0] Row
        [0] Column
            [0] Markdown(str, sizing_mode='stretch_width', styles={'background': '...})
            [1] Param(MainApplication, show_name=False, sizing_mode='stretch_width', styles={'background': '...}, widgets={'username': {'widget_type...})
            [2] ParamMethod(method, _pane=Column, defer_load=False)
        [1] HTML(str)